In [90]:
import pandas as pd
import numpy as np
import time
import datetime
import random as rd
from IPython.display import clear_output

START_MONTH = 11
END_MONTH = 12 # 12월로 고정추천
DAY_NUM = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
rent_file_name = ["", "", "", "", "", "", "",
                  "서울특별시 공공자전거 대여이력 정보_2207.csv", 
                  "서울특별시 공공자전거 대여이력 정보_22081.csv",
                  "서울특별시 공공자전거 대여이력 정보_2209.csv",
                  "서울특별시 공공자전거 대여이력 정보_2210.csv", 
                  "서울특별시 공공자전거 대여이력 정보_2211.csv",
                  "서울특별시 공공자전거 대여이력 정보_2212.csv"]

In [91]:
# 데이터 로드

df_rent = [None] * 13
for i in range(START_MONTH, END_MONTH + 1):
    df_rent[i] = pd.read_csv(rent_file_name[i], encoding='cp949')
    print(f"{i}월 대여이력 로드 완료")

df_broken = pd.read_csv("서울시 공공자전거 고장신고 내역_22.07-12.csv", encoding='cp949')
print("고장데이터 로드 완료")

11월 대여이력 로드 완료
12월 대여이력 로드 완료
고장데이터 로드 완료


In [93]:
def 대여이력정보_전처리(df):
    # 시계열 데이터 전처리
    df = df[(df['대여일시'] != '0000-00-00 00:00:00') & (df['반납일시'] != '0000-00-00 00:00:00')]
    df['대여일시'] = pd.to_datetime(df['대여일시']) # 문자열로 되있는 날짜들을 datatime 타입으로 변경
    df['대여요일'] = df['대여일시'].dt.day_name()
    df['반납일시'] = pd.to_datetime(df['반납일시'])
    
    # 반납이 되지 않아 대여소 번호가 없으면 -1로 전처리
    df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')
    df["반납대여소번호"] = df["반납대여소번호"].str.replace('N','-1')
    df = df.astype({'반납대여소번호': 'int'})
    
     # 중복되어 필요없는 열 삭제
    df.drop(['대여거치대', '반납거치대', '대여대여소ID', '반납대여소ID'], axis=1, inplace=True)
    return df

def 고장신고내역_전처리(df):
    df['등록일시'] = pd.to_datetime(df['등록일시']) # 문자열로 되있는 날짜들을 datatime 타입으로 변경
    df = df.drop_duplicates(['자전거번호', '등록일시']) # 같은 고장신고인데 유형만 다른 데이터 중복 제거
    return df

df_broken_p = 고장신고내역_전처리(df_broken)
print("고장신고내역 전처리 완료")

# 대여이력 전처리 실행
df_rent_m = [None] * 13 # 월 기준으로 나눈 대여이력 : df_rent_m[12] -> 12월 대여이력 df 
for m in range(START_MONTH, END_MONTH + 1):
    df_rent_m[m] = 대여이력정보_전처리(df_rent[m])
    print(f"{m}월 대여이력 전처리 완료")

고장신고내역 전처리 완료


C:\Users\imihl\AppData\Local\Temp\ipykernel_15236\3570315090.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')


11월 대여이력 전처리 완료


C:\Users\imihl\AppData\Local\Temp\ipykernel_15236\3570315090.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["반납대여소번호"] = df["반납대여소번호"].str.replace('\\','')


12월 대여이력 전처리 완료


In [94]:
# 필요한 날짜만 조회하여 시간을 줄이기 위해 일 단위로 df 나누기
df_rent_d = [[None] * 32 for i in range(13)] # 월, 일 기준으로 나눈 대여이력 : df_rent_d[12][25] -> 12월 25일 대여이력 df
for m in range(START_MONTH, END_MONTH + 1):
    now = datetime.datetime(2022, m, 1)
    now = now.replace(hour=0, minute=0, second=0)
    next_day = now + timedelta(hours=24)
    
    for d in range(1, DAY_NUM[m] + 1):
        df_rent_d[m][d] = df_rent_m[m][(now <= df_rent_m[m]['대여일시']) & (df_rent_m[m]['대여일시'] < next_day)]
        now += timedelta(hours=24)
        next_day += timedelta(hours=24)

print(df_rent_d[START_MONTH][5].head())

            자전거번호                대여일시  대여 대여소번호       대여 대여소명  \
614365  SPB-36412 2022-11-05 00:00:32      1112   마곡엠밸리4단지 정문   
614366  SPB-50709 2022-11-05 00:01:26      2033  사당동 아르테스 웨딩앞   
614367  SPB-57225 2022-11-05 00:00:12      1251      석촌역 2번출구   
614368  SPB-46458 2022-11-05 00:01:51       114  홍대입구역 8번출구 앞   
614369  SPB-84181 2022-11-05 00:02:53       577      광진청소년수련관   

                      반납일시  반납대여소번호        반납대여소명  이용시간(분)  이용거리(M)    생년  성별  \
614365 2022-11-05 00:03:02     2702     마곡 엠밸리2단지        2     0.00  1970   M   
614366 2022-11-05 00:03:16     2082   7호선 이수역7번출구        1   320.00  1980  \N   
614367 2022-11-05 00:03:16     1251      석촌역 2번출구        3    31.88  1965  \N   
614368 2022-11-05 00:03:18      114  홍대입구역 8번출구 앞        1    40.00  1965   F   
614369 2022-11-05 00:03:22      577      광진청소년수련관        0     0.00  2003   F   

       이용자종류      대여요일  
614365   내국인  Saturday  
614366   내국인  Saturday  
614367   내국인  Saturday  
614368   내국인  Saturday

In [ ]:
def 고장신고부터_n시간전까지_대여이력추출(n, start_month = 12):
    rent_num = [] # n시간동안 대여횟수
    broken_id = [] # 원본 파일 인덱스
    broken_date = []
    
    df_new_broken = pd.DataFrame(columns=df_broken_p.columns)
    df_rent_broken = pd.DataFrame(columns = df_rent_m[start_month].columns)
    
    start_time = time.time()
    for i, row in df_broken_p.iterrows():
        end = row['등록일시']
        start = end - timedelta(hours=n)
        
        # start_month 월 이전 데이터를 사용해야하는 고장신고는 건너뛰기
        if start.month < start_month:
            continue

        filtered_df = pd.DataFrame(columns = df_rent_m[start_month].columns)
        # start와 end month가 다를수도 있으므로 해당 기간에 속한 month들 모두 탐색
        for m in range(start.month, end.month + 1):
            end_day = end.day if m == end.month else DAY_NUM[m]
            
            for d in range(start.day, end_day + 1):
            # 기간안에 속한 대여이력 추출
                tmp_df = df_rent_d[m][d][(start <= df_rent_d[m][d]['대여일시']) & (df_rent_d[m][d]['대여일시'] < end)]
                tmp_df = tmp_df[tmp_df['자전거번호'] == row[0]]
                filtered_df = pd.concat([tmp_df, filtered_df])
                

        rent_num.append(len(filtered_df))
        broken_id.append(i)
        broken_date.append(row['등록일시'])
        df_new_broken = pd.concat([df_new_broken, row.to_frame().T], ignore_index=True)
        filtered_df['고장일시'] = row['등록일시']
        df_rent_broken = pd.concat([df_rent_broken, filtered_df], ignore_index=True)
        
        if i % 100 == 0:
            clear_output()
            now = time.time() - start_time
            print("{:.2f}%... {:.2f}초 경과".format((i - broken_id[0]) / (len(df_broken) - broken_id[0]) * 100, now))
        
        
    print(f"100% 완료!... {time.time() - start_time}초 경과")
    
    name = str(n) + "시간동안대여수"
    df_new_broken[name] = rent_num
    df_new_broken["원본파일인덱스"] = broken_id
    
    name = f"{start_month}~12 고장신고내역(N={n}시간).csv"
    df_new_broken.to_csv(name, encoding='cp949')
    name = f"{start_month}~12 대여이력(N={n}시간내고장).csv"
    df_rent_broken.to_csv(name, encoding='cp949')
    print("파일저장완료")
    
    return df_new_broken, df_rent_broken

df, df2 = 고장신고부터_n시간전까지_대여이력추출(72, START_MONTH)

52.93%... 1585.69초 경과


In [85]:
# n = 시간
# short_dist = 짧은 대여 이력의 이용거리 기준
# short_time = 짧은 대여 이력의 이용시간 기준
# start_month = start_month 이후의 데이터만 다룸 - 7~12월을 다 하면 시간이 너무 들기 때문에 12월만 하는것으로 디폴트설정
def 고장신고부터_n시간전까지_대여이력추출(n, short_dist = 100, short_time = 1, start_month = 12):
    rent_num = [] # n시간동안 대여횟수
    short_rent_num = [] # n시간동안 짧은 대여 횟수
    broken_id = [] # 원본 파일 인덱스
    pro = [] # 짧은대여수 / 총 대여수
    broken_date = []
    
    df_new_broken = pd.DataFrame(columns=df_broken_p.columns)
    df_rent_broken = pd.DataFrame(columns = df_rent_m[start_month].columns)
    
    start_time = time.time()
    for i, row in df_broken_p.iterrows():
        end = row['등록일시']
        start = end - timedelta(hours=n)
        
        # start_month 월 이전 데이터를 사용해야하는 고장신고는 건너뛰기
        if start.month < start_month:
            continue

        filtered_df = pd.DataFrame(columns = df_rent_m[start_month].columns)
        # start와 end month가 다를수도 있으므로 해당 기간에 속한 month들 모두 탐색
        for m in range(start.month, end.month + 1):
            end_day = end.day if m == end.month else DAY_NUM[m]
            
            for d in range(start.day, end_day + 1):
            # 기간안에 속한 대여이력 추출
                tmp_df = df_rent_d[m][d][(start <= df_rent_d[m][d]['대여일시']) & (df_rent_d[m][d]['대여일시'] < end)]
                tmp_df = tmp_df[tmp_df['자전거번호'] == row[0]]
                filtered_df = pd.concat([tmp_df, filtered_df])
        rent_num.append(len(filtered_df))
        
        
        # 짧은 대여 이력 추출 - 거리와 시간 중 하나라도 만족하면 짧은 거리라고 판단
        filtered_df = filtered_df[(filtered_df['이용거리(M)'] <= short_dist) | (filtered_df['이용시간(분)'] <= short_time)]
        short_rent_num.append(len(filtered_df))
        pro.append(0 if rent_num[-1] == 0 else short_rent_num[-1] / rent_num[-1])

        broken_id.append(i)
        broken_date.append(row['등록일시'])
        df_new_broken = pd.concat([df_new_broken, row.to_frame().T], ignore_index=True)
        filtered_df['고장일시'] = row['등록일시']
        df_rent_broken = pd.concat([df_rent_broken, filtered_df], ignore_index=True)
        
        if i % 100 == 0:
            clear_output()
            now = time.time() - start_time
            print("{:.2f}%... {:.2f}초 경과".format((i - broken_id[0]) / (len(df_broken) - broken_id[0]) * 100, now))
        
        
    print(f"100% 완료!... {time.time() - start_time}초 경과")
    
    name = str(n) + "시간동안대여수"
    df_new_broken[name] = rent_num
    df_new_broken["짧은대여수"] = short_rent_num
    df_new_broken["짧은대여비율"] = pro
    df_new_broken["원본파일인덱스"] = broken_id
    
    name = f"{start_month}~12 고장신고내역+짧은대여(N={n}시간,dist={short_dist},time={short_time}).csv"
    df_new_broken.to_csv(name, encoding='cp949')
    name = f"{start_month}~12 짧은대여이력(N={n}시간,dist={short_dist},time={short_time}).csv"
    df_rent_broken.to_csv(name, encoding='cp949')
    print("파일저장완료")
    
    return df_new_broken, df_rent_broken

df, df2 = 고장신고부터_n시간전까지_대여이력추출(72)

KeyboardInterrupt: 

In [ ]:
df_rent_m[12]

def 정상자전거_대여이력추출(n, size):
    a = 0